In [67]:
import cv2
import mediapipe as mp
import time
import mediapipe.framework.formats.landmark_pb2 as landmark_pb2

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose
i=0
landmarks1 = landmark_pb2.NormalizedLandmarkList()


#get a class name otherwise assign classname as working
# class_name = input("Enter a Pose for class column")
# if class_name == '':
#     class_name = "working"

#adds a countdown to get user in position for pose collection
t=2
print("running pose collection in \n \tpress Esc to close")
while t > 0:
        print(t)
        t -= 1
        time.sleep(1)

# For webcam input:
sets = 0
cap = cv2.VideoCapture(0)
predicted = ''
with mp_pose.Pose(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as pose:
  while cap.isOpened():
    success, image = cap.read()
    if not success:
      print("Ignoring empty camera frame.")
      predicted = 'empty'
      # If loading a video, use 'break' instead of 'continue'.
      continue

    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pose.process(image)
    cv2.putText(image, predicted, (300, 100), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 3, cv2.LINE_AA)

#     time.sleep(1)
    
    # Draw the pose annotation on the image.
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)


#     text_flipped = cv2.flip(image, 1)
#     image = cv2.flip(image, 1)
    mp_drawing.draw_landmarks(
            image,
        results.pose_landmarks,
        mp_pose.POSE_CONNECTIONS,
        landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
    # Flip the image horizontally for a selfie-view display.
#     image = cv2.flip(image, 1)
    cv2.imshow('MediaPipe Pose', cv2.flip(image, 1))
    cv2.setWindowProperty("MediaPipe Pose", cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)

    i+=1
    if (results.pose_landmarks is not None):
        if i >=20:
#         write_dict_list_to_csv(results.pose_landmarks.landmark, class_name, 'landmarks.csv')
            predicted = predict(results.pose_landmarks.landmark)
#             print(results.pose_landmarks)
            sets+=1
            print(f"sets collected {sets}")
            i=0
#     print(results.pose_landmarks)
    if cv2.waitKey(5) & 0xFF == 27:
      break
cap.release()
cv2.destroyWindow('MediaPipe Pose')

running pose collection in 
 	press Esc to close
2
1
1/1 [==============================] - 0s 25ms/step
sets collected 1
1/1 [==============================] - 0s 28ms/step
sets collected 2
1/1 [==============================] - 0s 39ms/step
sets collected 3
1/1 [==============================] - 0s 22ms/step
sets collected 4
1/1 [==============================] - 0s 24ms/step
sets collected 5
1/1 [==============================] - 0s 19ms/step
sets collected 6
1/1 [==============================] - 0s 28ms/step
sets collected 7
1/1 [==============================] - 0s 86ms/step
sets collected 8
1/1 [==============================] - 0s 38ms/step
sets collected 9
1/1 [==============================] - 0s 23ms/step
sets collected 10
1/1 [==============================] - 0s 19ms/step
sets collected 11
1/1 [==============================] - 0s 37ms/step
sets collected 12
1/1 [==============================] - 0s 22ms/step
sets collected 13
1/1 [==============================] - 0s 33ms

In [21]:
cap.release()
cv2.destroyWindow('MediaPipe Pose')

In [63]:
import pandas as pd

def convert_landmarks_to_dataframe(landmarks):
    # Check if the landmarks list is empty or contains None values
    if not landmarks or None in landmarks:
        return pd.DataFrame()
    
    # Create a dictionary with the values of each NormalizedLandmark object in the list
    row = {}
    for i in range(len(landmarks)):
        normalized_landmark = landmarks[i]
        row[f"x{i+1}"] = normalized_landmark.x
        row[f"y{i+1}"] = normalized_landmark.y
        row[f"z{i+1}"] = normalized_landmark.z
        row[f"visibility{i+1}"] = normalized_landmark.visibility
    
    # Create a Pandas dataframe from the dictionary
    df = pd.DataFrame([row])
    return df

In [64]:
convert_landmarks_to_dataframe(results.pose_landmarks.landmark)

,x1,y1,z1,visibility1,x2,y2,z2,visibility2,x3,y3,...,z31,visibility31,x32,y32,z32,visibility32,x33,y33,z33,visibility33
0,0.691393,0.918563,-1.585427,0.99655,0.723314,0.857939,-1.53929,0.994406,0.741288,0.861298,...,0.366948,0.000131,0.63705,3.272864,-0.286246,0.000288,0.475673,3.228985,-0.342598,0.00027


In [7]:
import csv
import pandas as pd

def write_dict_list_to_csv(landmark_list):
  # Open the CSV file for writing in append mode
  
    fieldnames = []
    for i in range(len(landmark_list)):
      fieldnames.extend([f"x{i+1}", f"y{i+1}", f"z{i+1}", f"visibility{i+1}"])
    # Add the class name as the last field
    fieldnames.append('class')
    

    # Create a dictionary with the values of each NormalizedLandmark object in the list
    row = {}
    for i in range(len(landmark_list)):
      normalized_landmark = landmark_list[i]
      row[f"x{i+1}"] = normalized_landmark.x
      row[f"y{i+1}"] = normalized_landmark.y
      row[f"z{i+1}"] = normalized_landmark.z
      row[f"visibility{i+1}"] = normalized_landmark.visibility
    # Add the class name to the row
    # Write the dictionary as a row in the CSV file

  
  
    data = row
    new = pd.DataFrame.from_dict(data, orient ='index')
    return new

In [6]:
x = write_dict_list_to_csv(results.pose_landmarks.landmark)

In [7]:
x.drop('class').values

array([[0.6344282031059265],
       [0.710940420627594],
       [-1.3856899738311768],
       [0.9999334812164307],
       [0.6581674814224243],
       [0.6617523431777954],
       [-1.3716752529144287],
       [0.999833345413208],
       [0.6686854958534241],
       [0.6619142293930054],
       [-1.3711966276168823],
       [0.9998397827148438],
       [0.6776990294456482],
       [0.661994218826294],
       [-1.3712471723556519],
       [0.999764621257782],
       [0.6184664964675903],
       [0.6534399390220642],
       [-1.3940351009368896],
       [0.99989253282547],
       [0.60341477394104],
       [0.6478306651115417],
       [-1.393911361694336],
       [0.9998992085456848],
       [0.5890911221504211],
       [0.6420803070068359],
       [-1.3939545154571533],
       [0.9998447299003601],
       [0.6741799116134644],
       [0.6650322079658508],
       [-1.1173044443130493],
       [0.9998070597648621],
       [0.5490282773971558],
       [0.6350849866867065],
       [-1.2176

In [4]:
import tensorflow as tf
from tensorflow import keras
from keras.models import load_model

# Load the model
model = load_model('model1.h5')

2023-01-03 09:10:05.520052: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-03 09:10:05.735643: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/codered/.local/lib/python3.10/site-packages/cv2/../../lib64:
2023-01-03 09:10:05.735673: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-03 09:10:07.222057: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dler

In [55]:
x1 = pd.DataFrame(x.drop('class').iloc[0:].values.reshape(1, -1))

In [83]:
x1.iloc[:1]

,0,1,2,3,4,5,6,7,8,9,...,122,123,124,125,126,127,128,129,130,131
0,0.634428,0.71094,-1.38569,0.999933,0.658167,0.661752,-1.371675,0.999833,0.668685,0.661914,...,0.273447,0.000009,0.576066,2.334294,0.006472,0.000063,0.431789,2.297956,-0.143948,0.000024


In [84]:
model.predict(x2)

1/1 [==============================] - 0s 85ms/step


array([[4.798374e-08, 9.997719e-01, 6.379073e-04]], dtype=float32)

In [70]:
print(x1.values.dtype)

object


In [78]:
x2 = x1.apply(pd.to_numeric, errors='coerce', downcast='float')
x2.dtypes

0      float32
1      float32
2      float32
3      float32
4      float32
        ...   
127    float32
128    float32
129    float32
130    float32
131    float32
Length: 132, dtype: object

In [86]:
import numpy as np
# predictions = model.predict(df.iloc[:1].drop('class', axis=1))
index = 1000

# Create a dictionary that maps the class numbers to the class labels
class_labels = {0: 'away', 1: 'phone', 2: 'working'}

# Make a prediction
prediction = model.predict(x2)

# Find the index of the highest probability
class_index = np.argmax(prediction)

# Look up the class label in the dictionary
class_label = class_labels[class_index]

# Print the class label
print("Class label:", class_label)

# print("actual: ", df.iloc[index-1:index]['class'])

1/1 [==============================] - 0s 44ms/step
Class label: phone


In [65]:
import numpy as np

def predict(landmark):
    x = convert_landmarks_to_dataframe(landmark)
#     x = pd.DataFrame(x.iloc[0:].values.reshape(1, -1))
    x = x.apply(pd.to_numeric, errors='coerce', downcast='float')
    class_labels = {0: 'away', 1: 'phone', 2: 'working'}

    # Make a prediction
    prediction = model.predict(x)

    # Find the index of the highest probability
    class_index = np.argmax(prediction)

    # Look up the class label in the dictionary
    class_label = class_labels[class_index]

    # Print the class label
#     print("Class label:", class_label)
    return class_label

In [192]:
predict(results.pose_landmarks.landmark)

1/1 [==============================] - 0s 23ms/step


'working'

In [157]:
import cv2

# Start the webcam and get a video capture object
capture = cv2.VideoCapture(0)

while True:
  # Capture a frame from the webcam
  _, frame = capture.read()

  # Flip the frame horizontally
  frame = cv2.flip(frame, 1)

  # Draw the text on the frame
  cv2.putText(frame, "Hello, World!", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255, 255, 255), 2, cv2.LINE_AA)

  # Flip the text horizontally
#   frame = cv2.flip(frame, 1)

  # Display the frame
  cv2.imshow("Flipped Text", frame)

  # Wait for a key press to exit the loop
  key = cv2.waitKey(1)
  if key == ord("q"):
    break

# Release the video capture object and close all windows
capture.release()
cv2.destroyAllWindows()
